### Python Task

#### ETL Pipeline

In [1]:
import pandas as pd

In [2]:
# Step 1: Extract
df = pd.read_csv('product_reviews.csv')

In [3]:
# Step 2: Transform
# Remove rows with null rating or review_text
df = df.dropna(subset=['rating', 'review_text'])

In [4]:
# Convert rating to integer and filter out invalid ratings
df['rating'] = df['rating'].astype(int)
df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]

In [5]:
# Sentiment extraction logic
df['sentiment'] = df['review_text'].apply(lambda x: 'Negative' if 'bad' in x.lower() else 'Positive')

In [6]:
# Step 3: Load
df.to_csv('cleaned_product_reviews.csv', index=False)

#### Data Deduplication (Python)

In [7]:
# Step 1: Read the file
df = pd.read_csv('user_data.csv')

In [8]:
# Step 2: Data Deduplication
df['last_updated'] = pd.to_datetime(df['last_updated'])  # Ensure 'last_updated' is in datetime format
df = df.sort_values('last_updated', ascending=False)  # Sort by most recent update
df = df.drop_duplicates(subset='email', keep='first')  # Drop duplicates, keeping the most recent

In [9]:
# Step 3: Output the cleaned file
df.to_csv('cleaned_user_data.csv', index=False)

### SQL Task

In [10]:
import sqlite3

# Connect to an in-memory SQLite database (for temporary work)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [11]:
# Example: creating users and orders
users_df = pd.DataFrame({
    'user_id': ['U1', 'U2', 'U3'],
    'name': ['Alice', 'Bob', 'Charlie'],
    'signup_date': ['2023-01-01', '2023-02-01', '2023-03-01']
})

orders_df = pd.DataFrame({
    'order_id': ['O1', 'O2', 'O3', 'O4', 'O5', 'O6'],
    'user_id': ['U1', 'U1', 'U1', 'U1', 'U2', 'U3'],
    'amount': [100, 150, 200, 50, 80, 40],
    'order_date': ['2025-02-01', '2025-02-15', '2025-03-01', '2025-04-01', '2025-01-15', '2025-04-01']
})

users_df.to_sql('users', conn, index=False, if_exists='replace')
orders_df.to_sql('orders', conn, index=False, if_exists='replace')


6

#### 1. Users with >3 orders in last 90 days

In [12]:
query1 = '''
SELECT 
    u.user_id,
    u.name,
    COUNT(o.order_id) AS order_count,
    SUM(o.amount) AS total_spent
FROM 
    users u
JOIN 
    orders o ON u.user_id = o.user_id
WHERE 
    o.order_date >= DATE('now', '-90 day')
GROUP BY 
    u.user_id, u.name
HAVING 
    COUNT(o.order_id) > 3;
'''

df_result1 = pd.read_sql_query(query1, conn)
print(df_result1)


  user_id   name  order_count  total_spent
0      U1  Alice            4          500


### Missing Hourly Sensor Logs

In [13]:
# Sample data first
sensor_df = pd.DataFrame({
    'sensor_id': ['S1'] * 23 + ['S2'] * 24,
    'timestamp': pd.date_range(end=pd.Timestamp.now(), periods=23, freq='H').tolist() +
                 pd.date_range(end=pd.Timestamp.now(), periods=24, freq='H').tolist(),
    'reading': [42]*47
})
sensor_df.to_sql('sensor_logs', conn, index=False, if_exists='replace')

# Run the missing data query
query2 = '''
WITH hours AS (
    SELECT DATETIME(DATETIME('now', '-1 day'), '+' || n || ' hours') AS expected_time
    FROM (
        SELECT 0 AS n UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4
        UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9
        UNION ALL SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14
        UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19
        UNION ALL SELECT 20 UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23
    )
),
sensors AS (
    SELECT DISTINCT sensor_id FROM sensor_logs
),
expected_logs AS (
    SELECT 
        s.sensor_id, 
        h.expected_time
    FROM 
        sensors s
    CROSS JOIN 
        hours h
)
SELECT 
    e.sensor_id, 
    e.expected_time
FROM 
    expected_logs e
LEFT JOIN 
    sensor_logs sl 
    ON e.sensor_id = sl.sensor_id 
    AND strftime('%Y-%m-%d %H', sl.timestamp) = strftime('%Y-%m-%d %H', e.expected_time)
WHERE 
    sl.timestamp IS NULL
ORDER BY 
    e.sensor_id, e.expected_time;
'''

df_result2 = pd.read_sql_query(query2, conn)
print(df_result2)


   sensor_id        expected_time
0         S1  2025-04-11 10:20:55
1         S1  2025-04-11 11:20:55
2         S1  2025-04-11 12:20:55
3         S1  2025-04-11 13:20:55
4         S1  2025-04-11 14:20:55
5         S1  2025-04-11 15:20:55
6         S1  2025-04-11 16:20:55
7         S2  2025-04-11 10:20:55
8         S2  2025-04-11 11:20:55
9         S2  2025-04-11 12:20:55
10        S2  2025-04-11 13:20:55
11        S2  2025-04-11 14:20:55
12        S2  2025-04-11 15:20:55


###  VIP Gold Customers (Purchases + Membership)

In [14]:
# Sample data
purchases_df = pd.DataFrame({
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004'],
    'customer_id': ['C001', 'C002', 'C001', 'C003'],
    'amount': [120.50, 200.00, 300.00, 50.00],
    'transaction_date': ['2024-03-15', '2024-03-15', '2024-03-16', '2024-03-17']
})

members_df = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C004'],
    'member_since': ['2023-05-01', '2023-11-15', '2024-01-10'],
    'membership_level': ['Gold', 'Silver', 'Gold']
})

purchases_df.to_sql('purchases', conn, index=False, if_exists='replace')
members_df.to_sql('members', conn, index=False, if_exists='replace')

query3 = '''
SELECT 
    m.customer_id,
    SUM(p.amount) AS total_spent,
    m.membership_level
FROM 
    purchases p
INNER JOIN 
    members m ON p.customer_id = m.customer_id
WHERE 
    m.membership_level = 'Gold'
GROUP BY 
    m.customer_id, m.membership_level
HAVING 
    total_spent > 250;
'''

df_result3 = pd.read_sql_query(query3, conn)
print(df_result3)


  customer_id  total_spent membership_level
0        C001        420.5             Gold
